# Can I append an `xr.Dataset` full of `NaN`s where there is a gap in the time series?
This would be useful for gracefully building a time-contiguous Zarr

In [83]:
import os
import iris
import xarray as xr

filepath = '/data/cssp-china/sample-data-17-01-20/cssp_china_pp'
filename = 'apepda.pa508i0.pp'
cube = iris.load_cube(os.path.join(filepath, filename), 'precipitation_flux')
display(cube)
ds = xr.DataArray.from_iris(cube).to_dataset()
ds = ds.chunk(chunks={'time':10, 'grid_latitude':219, 'grid_longitude':286})
display(ds)

/envs/auto-build-envs/datasci/lib/python3.6/site-packages/iris/analysis/_interpolation.py:300: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data = data[dim_slices]


<iris 'Cube' of precipitation_flux / (kg m-2 s-1) (time: 10; grid_latitude: 219; grid_longitude: 286)>

<xarray.Dataset>
Dimensions:                  (grid_latitude: 219, grid_longitude: 286, time: 10)
Coordinates:
  * time                     (time) datetime64[ns] 1850-08-08T12:00:00 ... 1850-08-17T12:00:00
  * grid_latitude            (grid_latitude) float32 22.88 22.66 ... -25.08
  * grid_longitude           (grid_longitude) float32 323.48 323.7 ... 386.18002
    forecast_reference_time  datetime64[ns] ...
    forecast_period          (time) timedelta64[ns] dask.array<chunksize=(10,), meta=np.ndarray>
Data variables:
    precipitation_flux       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(10, 219, 286), meta=np.ndarray>

In [85]:
ds.precipitation_flux.data

dask.array<rechunk-merge, shape=(10, 219, 286), dtype=float32, chunksize=(10, 219, 286), chunktype=numpy.ndarray>

In [86]:
import dask.array as da
import numpy as np
narr = da.full_like(ds_z.precipitation_flux.data.blocks[0], np.nan)
narr

dask.array<full, shape=(10, 219, 286), dtype=float32, chunksize=(10, 219, 286), chunktype=numpy.ndarray>

In [68]:
narr.compute()

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan

In [88]:
ds.precipitation_flux.time.data

array(['1850-08-08T12:00:00.000000000', '1850-08-09T12:00:00.000000000',
       '1850-08-10T12:00:00.000000000', '1850-08-11T12:00:00.000000000',
       '1850-08-12T12:00:00.000000000', '1850-08-13T12:00:00.000000000',
       '1850-08-14T12:00:00.000000000', '1850-08-15T12:00:00.000000000',
       '1850-08-16T12:00:00.000000000', '1850-08-17T12:00:00.000000000'],
      dtype='datetime64[ns]')

## Sythesise a new time coord

In [91]:
import pandas as pd
dates = pd.date_range(start='1850-08-18 12:00:00.000', end='1850-08-31 12:00:00.000', freq='1d')
dates

DatetimeIndex(['1850-08-18 12:00:00', '1850-08-19 12:00:00',
               '1850-08-20 12:00:00', '1850-08-21 12:00:00',
               '1850-08-22 12:00:00', '1850-08-23 12:00:00',
               '1850-08-24 12:00:00', '1850-08-25 12:00:00',
               '1850-08-26 12:00:00', '1850-08-27 12:00:00',
               '1850-08-28 12:00:00', '1850-08-29 12:00:00',
               '1850-08-30 12:00:00', '1850-08-31 12:00:00'],
              dtype='datetime64[ns]', freq='D')

In [102]:
new_time = np.array(dates)
new_time

array(['1850-08-18T12:00:00.000000000', '1850-08-19T12:00:00.000000000',
       '1850-08-20T12:00:00.000000000', '1850-08-21T12:00:00.000000000',
       '1850-08-22T12:00:00.000000000', '1850-08-23T12:00:00.000000000',
       '1850-08-24T12:00:00.000000000', '1850-08-25T12:00:00.000000000',
       '1850-08-26T12:00:00.000000000', '1850-08-27T12:00:00.000000000',
       '1850-08-28T12:00:00.000000000', '1850-08-29T12:00:00.000000000',
       '1850-08-30T12:00:00.000000000', '1850-08-31T12:00:00.000000000'],
      dtype='datetime64[ns]')

## Make a copy of the `xr.Dataset` and change the time coord data

In [99]:
ds_cp = ds.copy(deep=True, data={'precipitation_flux':narr})
ds_cp

<xarray.Dataset>
Dimensions:                  (grid_latitude: 219, grid_longitude: 286, time: 10)
Coordinates:
  * time                     (time) datetime64[ns] 1850-08-08T12:00:00 ... 1850-08-17T12:00:00
  * grid_latitude            (grid_latitude) float32 22.88 22.66 ... -25.08
  * grid_longitude           (grid_longitude) float32 323.48 323.7 ... 386.18002
    forecast_reference_time  datetime64[ns] 1849-12-01
    forecast_period          (time) timedelta64[ns] dask.array<chunksize=(10,), meta=np.ndarray>
Data variables:
    precipitation_flux       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(10, 219, 286), meta=np.ndarray>

In [101]:
ds_cp.precipitation_flux.compute()

<xarray.DataArray 'precipitation_flux' (time: 10, grid_latitude: 219, grid_longitude: 286)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, n

In [104]:
ds_cp.time.data = new_time[0:10]

In [106]:
ds_cp.precipitation_flux

<xarray.DataArray 'precipitation_flux' (time: 10, grid_latitude: 219, grid_longitude: 286)>
dask.array<full, shape=(10, 219, 286), dtype=float32, chunksize=(10, 219, 286), chunktype=numpy.ndarray>
Coordinates:
  * time                     (time) datetime64[ns] 1850-08-18T12:00:00 ... 1850-08-27T12:00:00
  * grid_latitude            (grid_latitude) float32 22.88 22.66 ... -25.08
  * grid_longitude           (grid_longitude) float32 323.48 323.7 ... 386.18002
    forecast_reference_time  datetime64[ns] 1849-12-01
    forecast_period          (time) timedelta64[ns] dask.array<chunksize=(10,), meta=np.ndarray>
Attributes:
    standard_name:  precipitation_flux
    units:          kg m-2 s-1
    source:         Data from Met Office Unified Model
    STASH:          m01s05i216
    cell_methods:   time: mean (interval: 1 hour)

In [107]:
ds_cp2 = ds.copy(deep=True, data={'time': new_time, 'precipitation_flux':narr})
ds_cp

ValueError: Data must only contain variables in original dataset. Extra variables: {'time'}

### OK so method meeds to be:
1. Deep copy of dataset, with data replacement
2. Replace data in time coordinate with new times

## Now let's try to append the new `xr.Dataset` to the Zarr

In [108]:
zarrpath = 'zarr_precip'
ds_cp.to_zarr(zarrpath, consolidated=True, append_dim='time')

In [109]:
xr.open_zarr(zarrpath)

<xarray.Dataset>
Dimensions:                  (grid_latitude: 219, grid_longitude: 286, time: 30)
Coordinates:
    forecast_period          (time) timedelta64[ns] dask.array<chunksize=(10,), meta=np.ndarray>
    forecast_reference_time  datetime64[ns] ...
  * grid_latitude            (grid_latitude) float32 22.88 22.66 ... -25.08
  * grid_longitude           (grid_longitude) float32 323.48 323.7 ... 386.18002
  * time                     (time) datetime64[ns] 1850-08-08T12:00:00 ... 1850-08-27T12:00:00
Data variables:
    precipitation_flux       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(10, 219, 286), meta=np.ndarray>

## Success!